# Neural Network from Scratch — Letter Classifier (A, B, C)

This notebook implements a small feedforward neural network (one hidden layer) **from scratch** using only NumPy to classify binary 5×6 pixel patterns representing the letters **A**, **B**, and **C**. It trains the network with backpropagation and visualizes loss, accuracy, and predictions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# For reproducibility
np.random.seed(42)


In [ ]:
# Each letter is a 5 rows x 6 cols pattern (5x6 = 30 pixels). 1 = ink / white pixel, 0 = background
A = [
    [0,1,1,1,0,0],
    [1,0,0,0,1,0],
    [1,1,1,1,1,0],
    [1,0,0,0,1,0],
    [1,0,0,0,1,0]
]

B = [
    [1,1,1,1,0,0],
    [1,0,0,0,1,0],
    [1,1,1,1,0,0],
    [1,0,0,0,1,0],
    [1,1,1,1,0,0]
]

C = [
    [0,1,1,1,1,0],
    [1,0,0,0,0,1],
    [1,0,0,0,0,0],
    [1,0,0,0,0,1],
    [0,1,1,1,1,0]
]

A = np.array(A).reshape(30)
B = np.array(B).reshape(30)
C = np.array(C).reshape(30)

X = np.vstack([A, B, C]).astype(np.float32)
y = np.array([[1,0,0], [0,1,0], [0,0,1]], dtype=np.float32)

print('X shape:', X.shape)
print('y shape:', y.shape)


In [ ]:
# Network hyperparameters
input_size = 30
hidden_size = 8   # you can tune this
output_size = 3
learning_rate = 0.5
epochs = 5000

# Weight initialization (small random values)
W1 = np.random.randn(input_size, hidden_size) * 0.1
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.1
b2 = np.zeros((1, output_size))

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(sig_x):
    # input is sigmoid(x) already (common trick)
    return sig_x * (1 - sig_x)


In [ ]:
loss_history = []
acc_history = []

for epoch in range(1, epochs+1):
    # Forward pass
    z1 = np.dot(X, W1) + b1            # (3, hidden_size)
    a1 = sigmoid(z1)                   # hidden activations
    z2 = np.dot(a1, W2) + b2           # (3, output_size)
    a2 = sigmoid(z2)                   # output activations

    # Loss (MSE) across the batch
    loss = np.mean((y - a2) ** 2)
    loss_history.append(loss)

    # Accuracy (simple argmax)
    preds = np.argmax(a2, axis=1)
    labels = np.argmax(y, axis=1)
    acc = np.mean(preds == labels)
    acc_history.append(acc)

    # Backpropagation
    error_output = (y - a2)               # (3, output_size)
    delta_output = error_output * sigmoid_derivative(a2)

    error_hidden = np.dot(delta_output, W2.T)   # (3, hidden_size)
    delta_hidden = error_hidden * sigmoid_derivative(a1)

    # Gradients (batch gradients)
    dW2 = np.dot(a1.T, delta_output)            # (hidden_size, output_size)
    db2 = np.sum(delta_output, axis=0, keepdims=True)
    dW1 = np.dot(X.T, delta_hidden)             # (input_size, hidden_size)
    db1 = np.sum(delta_hidden, axis=0, keepdims=True)

    # Update weights (gradient ascent on squared error: we add because error_output was (y - a2))
    W2 += learning_rate * dW2
    b2 += learning_rate * db2
    W1 += learning_rate * dW1
    b1 += learning_rate * db1

    if epoch % 500 == 0 or epoch == 1:
        print(f'Epoch {epoch}/{epochs} — loss: {loss:.6f} — acc: {acc:.3f}')


In [ ]:
plt.figure(figsize=(8,4))
plt.plot(loss_history)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Loss over Epochs')
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(8,4))
plt.plot(acc_history)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy over Epochs')
plt.grid(True)
plt.show()


In [ ]:
def predict(sample):
    z1 = np.dot(sample.reshape(1, -1), W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)
    return a2.flatten()

labels = ['A', 'B', 'C']
for i, s in enumerate(X):
    out = predict(s)
    pred_idx = np.argmax(out)
    print(f'True: {labels[i]}  — Pred: {labels[pred_idx]}  — Probabilities: {out.round(3)}')
    plt.figure(figsize=(2,3))
    plt.imshow(s.reshape(5,6), cmap='gray')
    plt.title(f'True: {labels[i]}  Pred: {labels[pred_idx]}')
    plt.axis('off')
    plt.show()


In [ ]:
# Save trained weights (so you can load later)
out_dir = Path('/mnt/data')
np.savez(out_dir / 'nn_weights.npz', W1=W1, b1=b1, W2=W2, b2=b2)
print('Saved weights to', out_dir / 'nn_weights.npz')


## Conclusion

This simple network shows how to implement forward and backward passes using NumPy and train a tiny classifier on handcrafted binary images. For better generalization, you can:
- add more training samples (noisy variants),
- experiment with different hidden layer sizes and learning rates,
- use cross-entropy + softmax for the final layer,
- use regularization or momentum.
